# Hyperparameter Optimization and Ensemble models

En el ámbito del aprendizaje automático, **la optimización de hiperparámetros** y los **métodos de ensamblaje** son dos enfoques fundamentales que permiten mejorar significativamente el rendimiento de los modelos.

## Optimización de Hiperparámetros

Los modelos de aprendizaje automático dependen de ciertos parámetros que no se aprenden a partir de los datos, conocidos como **hiperparámetros**. Estos controlan aspectos como la estructura del modelo o la forma en que se entrena. La selección adecuada de hiperparámetros puede ser crucial para el rendimiento del modelo. Técnicas como la **búsqueda en cuadrícula** (*Grid Search*), la **búsqueda aleatoria** (*Random Search*) y la más avanzada **optimización bayesiana** se utilizan comúnmente para encontrar las mejores combinaciones de hiperparámetros.

## Métodos de Ensamblaje

Por otro lado, los **métodos de ensamblaje** se basan en la idea de combinar múltiples modelos para mejorar la precisión y la robustez de las predicciones. Estos métodos aprovechan la diversidad de los modelos individuales para corregir sus errores

En este cuaderno, exploraremos ambos conceptos, centrándonos en cómo optimizar los hiperparámetros de los modelos y cómo combinar múltiples modelos para obtener mejores resultados en tareas de predicción.


In [ ]:
# Manipulación de datos y visualización
import pandas as pd
import matplotlib.pyplot as plt

# División de datos y preprocesamiento
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Modelos básicos
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

# Métodos de ensamblaje (Ensemble)
from sklearn.ensemble import VotingClassifier, StackingClassifier, BaggingClassifier, AdaBoostClassifier

# Búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from skopt import BayesSearchCV

# Evaluación de modelos
from sklearn.metrics import f1_score

# Librería Lazy Predict (para comparación rápida de modelos)
from lazypredict.Supervised import LazyClassifier


En esta libreta usaremos el conjunto de datos que se utiliza para la predicción de la **calidad del vino**. Está compuesto por variables basadas en pruebas fisicoquímicas y datos sensoriales.

### Variables de entrada (pruebas fisicoquímicas):
1. **Fixed acidity** - Acidez fija
2. **Volatile acidity** - Acidez volátil
3. **Citric acid** - Ácido cítrico
4. **Residual sugar** - Azúcar residual
5. **Chlorides** - Cloruros
6. **Free sulfur dioxide** - Dióxido de azufre libre
7. **Total sulfur dioxide** - Dióxido de azufre total
8. **Density** - Densidad
9. **pH**
10. **Sulphates** - Sulfatos
11. **Alcohol**

### Variable de salida (basada en datos sensoriales):
12. **Quality** - Puntaje de calidad del vino (entre 0 y 10)

Este conjunto de datos permite construir modelos que predicen la calidad del vino en función de sus características fisicoquímicas.


In [ ]:
data = pd.read_csv('data/winequality-red.csv')

In [ ]:
data.head()

In [ ]:
data.describe()

Dividimos nuestro set de datos para hacer validación cruzada

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('quality', axis = 1), 
    data['quality'], 
    test_size=0.33, 
    random_state=42
)

Estandarizamos los datos

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

En la siguiente celda, utilizaremos **LazyClassifier** para entrenar varios modelos de clasificación con el conjunto de datos de calidad del vino.

In [ ]:
fast_models = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models,predictions = fast_models.fit(X_train, X_test, y_train, y_test)

models

## Optimización de parámetros

### K-nearest neighbors (Grid Search)

In [ ]:
parameters = {'n_neighbors':[2*x+1 for x in range(10)], 
              'weights':['uniform', 'distance'], 
              'algorithm':['ball_tree', 'kd_tree', 'brute'], 
              'metric':['minkowski', 'manhattan', 'euclidean']
             }
knn = KNeighborsClassifier()
grid_search_knn = GridSearchCV(knn, parameters, scoring = 'f1_micro', cv = 5, n_jobs = 2)

In [ ]:
grid_search_knn.fit(X_train, y_train)

In [ ]:
pd.DataFrame(grid_search_knn.cv_results_).head()

In [ ]:
grid_search_knn.best_params_

In [ ]:
knn_opt = grid_search_knn.best_estimator_

In [ ]:
preds = knn_opt.predict(X_test)
f1_score(y_test, preds, average='micro')

### Decision Tree (Random Search)

In [ ]:
param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': [x for x in range(5, 15)],
    'min_samples_split': [2, 3, 4, 5, 6],
    'min_samples_leaf': [1, 2, 3, 4],   
    'ccp_alpha': [0, 0.01, 0.1, 0.2],
}

In [ ]:
tree = DecisionTreeClassifier()
random_search_tree = RandomizedSearchCV(tree, param_grid, n_iter = 100, scoring = 'f1_micro', cv = 5, n_jobs = 2, random_state=42)

In [ ]:
random_search_tree.fit(X_train, y_train)

In [ ]:
random_search_tree.best_params_

In [ ]:
tree_opt = random_search_tree.best_estimator_

In [ ]:
preds = tree_opt.predict(X_test)
f1_score(y_test, preds, average='micro')

### Support Vector Machine (Bayesian Optimization)

In [ ]:
bayesian_opt = BayesSearchCV(
    SVC(),
    {
        'C': (1e-3, 1e+3, 'log-uniform'),
        'gamma': (1e-3, 1e+1, 'log-uniform'),
        'degree': (1, 6),
        'kernel': ['poly', 'rbf'], 
    },
    scoring = 'f1_micro',
    n_iter=21,
    n_points = 3,
    cv=5,
    n_jobs = 5,
    random_state = 42
    
)

In [ ]:
bayesian_opt.fit(X_train, y_train)

In [ ]:
bayesian_opt.best_params_

In [ ]:
svc_opt = bayesian_opt.best_estimator_

In [ ]:
preds = svc_opt.predict(X_test)
f1_score(y_test, preds, average='micro')

## Ensamble de modelos

### Por votación 

In [ ]:
eclf1 = VotingClassifier(estimators=[('knn', knn_opt), ('tree', tree_opt), ('SVC', svc_opt)], voting='hard')

In [ ]:
eclf1.fit(X_train, y_train)

In [ ]:
preds = eclf1.predict(X_test)
f1_score(y_test, preds, average='micro')

In [ ]:
svc_proba = SVC(C=19.958517651170755, gamma = 1.1270954208690063, kernel = 'rbf', probability = True)

In [ ]:
svc_proba.fit(X_train, y_train)

In [ ]:
eclf2 = VotingClassifier(estimators=[('knn', knn_opt), ('tree', tree_opt), ('SVC', svc_proba)], voting='soft')

In [ ]:
eclf2.fit(X_train, y_train)

In [ ]:
preds = eclf2.predict(X_test)
f1_score(y_test, preds, average='micro')

### Stacking

In [ ]:
eclf3 = StackingClassifier(
    estimators=[('knn', knn_opt), ('tree', tree_opt), ('SVC', svc_opt)], 
    final_estimator=SVC(kernel='linear')
)

In [ ]:
eclf3.fit(X_train, y_train)

In [ ]:
preds = eclf3.predict(X_test)
f1_score(y_test, preds, average='micro')

### Bagging

In [ ]:
eclf4 = BaggingClassifier(base_estimator=svc_opt, n_estimators=30, max_samples = 0.8, random_state=0).fit(X_train, y_train)

In [ ]:
preds = eclf4.predict(X_test)
f1_score(y_test, preds, average='micro')

## Ejercicio

In [ ]:
taxonomy = pd.read_csv('data/taxonomy_gut.csv', index_col = 0)

In [ ]:
taxonomy.T.head()

In [ ]:
metadata = pd.read_csv('data/metadata_gut.csv')

In [ ]:
metadata.head()

In [ ]:
resistance = pd.read_csv('data/ResistanceCiprofloxacinLoose.tsv.gz', compression='gzip', sep='\t')

In [ ]:
resistance['phenotype'].unique()